In [2]:
import pandas as pd
import os
import json
import requests
from pprint import pprint

In [3]:
def load_api_keys(file_path):
    """
    Load API keys from a JSON file.
    """
    with open(file_path, 'r') as file:
        keys = json.load(file)
    return keys

In [4]:
keys = load_api_keys('../data/api_keys.json')

In [5]:
def yelp_search(api_key, lat, lon):
    """
    Perform a Yelp search near the given latitude and longitude.
    """
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {'Authorization': f'Bearer {api_key}'}
    params = {
        'term': 'restaurant',
        'latitude': lat,
        'longitude': lon,
        'radius': 1609,  # Convert back to meters for Yelp's radius parameter
        'limit': 50  # Yelp API allows up to 50 results per request
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Error: Unable to fetch data from Yelp API. Status Code: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error
    

    data = response.json()

    # Create DataFrame from Yelp API response
    df = pd.DataFrame([{
        'Name': business['name'],
        'Business ID': business.get('id'),
        'Address': ', '.join(business['location'].get('display_address', [])),
        'Rating': business.get('rating'),
        'Total Ratings': business.get('review_count'),
        'Price': business.get('price', None),  # Price may not be available for all businesses
        'Cuisine': business.get('categories')
    } for business in data.get('businesses', [])])
    
    return df

In [6]:
df_points = pd.read_csv('../data/nola.csv', header=None, names=['Latitude', 'Longitude'],skiprows=1)

# Initialize an empty DataFrame to store Yelp results


In [7]:
df_points

,Latitude,Longitude
0,29.956970,-90.112957
1,30.011266,-90.077282
2,29.914587,-90.018549
3,29.975803,-90.046806
4,29.962644,-90.097513
...,...,...
102,29.909259,-90.003386
103,30.001888,-90.003923
104,30.018291,-90.036028
105,29.951151,-90.009855


In [8]:
combined_df = pd.DataFrame({
    'Name': ['existing restaurant'],
    'Business ID': ['id1'],
    'Address': ['100 Existing St'],
    'Rating': [0],
    'Total Ratings': [0],
    'Price': ['$']
})

In [9]:
for i, (index, row) in enumerate(df_points.iterrows()):
    if i >= 5:  # Stop after 5 iterations
        break    
    
    latitude, longitude = row['Latitude'], row['Longitude']

    print("hey",latitude,longitude)
    # Search Yelp using the latitude and longitude
    yelp_df = yelp_search(keys['yelp_api_key'], latitude, longitude)
    yelp_df['Name'] = yelp_df['Name'].str.lower()

    print(len(yelp_df))
    combined_df = pd.concat([combined_df, yelp_df[~yelp_df['Name'].isin(combined_df['Name'])]], ignore_index=True)

    i+=1


hey 29.956969553435577 -90.1129573456206
50
hey 30.011266383314485 -90.07728191476632
22
hey 29.91458695905421 -90.01854855648119
50
hey 29.975802650965527 -90.04680638143522
50
hey 29.96264442348628 -90.09751262655034
50


In [10]:
combined_df

,Name,Business ID,Address,Rating,Total Ratings,Price,Cuisine
0,existing restaurant,id1,100 Existing St,0.0,0,$,NaN
1,boucherie,rsMbMeW7IbXxPFh4WwbddQ,"8115 Jeannette St, New Orleans, LA 70118",4.3,627,$$,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia..."
2,ye olde college inn,yY3kq6_2m1FyY9MgaiHgoQ,"3000 S Carrollton Ave, New Orleans, LA 70118",4.2,272,$$,"[{'alias': 'southern', 'title': 'Southern'}, {..."
3,nice guys nola,AZJpX5slpbZUTU9INbwu0g,"7910 Earhart Blvd, New Orleans, LA 70125",3.2,459,$$,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':..."
4,loya's mexican restaurant,s4JWixWbeLhyUw1pEI2Cfg,"3102 Fern St, New Orleans, LA 70125",4.4,43,None,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias..."
...,...,...,...,...,...,...,...
190,anita's restaurant,kXbzA3rrsW32N78rrLovgA,"2122 Tulane Ave, New Orleans, LA 70112",4.0,92,$,"[{'alias': 'tradamerican', 'title': 'American'..."
191,juan's flying burrito,dGeXdSMah56gEHwZNaRQKA,"4724 S Carrollton Ave, New Orleans, LA 70119",3.6,565,$$,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a..."
192,good eden nola plantmade,2EOinQpyU4ChpyJ4KusO2A,"2940 Canal St, Ste A, New Orleans, LA 70119",4.6,156,$$,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias..."
193,mid-city yacht club,jrGaVL6KIwYpwivse_eKXw,"440 S St Patrick St, New Orleans, LA 70119",4.3,166,$$,"[{'alias': 'sportsbars', 'title': 'Sports Bars..."


In [11]:
import pymysql

import mysql.connector

In [25]:
host="vibez-database1-instance-1.c74gs48gc437.us-east-1.rds.amazonaws.com"
port=3306
dbname="restaurants"
user="admin"
password="Banker2b!"

conn = pymysql.connect(host=host, user=user,port=port,
                           password=password, database=dbname)


OperationalError: (2003, "Can't connect to MySQL server on 'vibez-database1-instance-1.c74gs48gc437.us-east-1.rds.amazonaws.com' (timed out)")